# Ice Versus Temperature and Salinity
Let's compare the ML-generated ice metrics to temperature and salinity data and see if there is any relationship.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import os

from cv_experiments.seg2info import Seg2Info
s2i = Seg2Info()
import cv2
import pickle

In [ ]:
root = "../cached/one"
images = [pickle.load(open(os.path.join(root, path), "rb")) for path in os.listdir(root) if path.endswith(".pkl")]

In [ ]:
import pandas as pd

td = pd.Timedelta(30, "s")  # Temperature and salinity data are in 1-minute increments
tabulars = os.scandir("../tabular/temp_salinity")
tabulars = filter(lambda f: f.is_file() and os.path.basename(f.path)[0] != "." and f.path[-4:] == ".csv", tabulars)
csv_dict = {os.path.basename(f.path).split("_")[0]:
    pd.read_csv(f.path, skiprows=[1], parse_dates=["time"]) for f in tabulars}

In [ ]:
def get_temp_salinity(name):
    boat = name.split("_")[0].replace("-", "")
    date = pd.to_datetime(name.split("_")[1], format="%Y-%m-%dT%H-%M-%S", utc=True)
    subset = csv_dict[boat]
    subset = subset[(subset["time"] >= date-td) & (subset["time"] < date+td)]
    temp = subset["TEMP_CTD_RBR_MEAN"].mean()
    salinity = subset["SAL_SBE37_MEAN"].mean()
    return temp, salinity

In [ ]:
s2i.apply(get_temp_salinity, images, "name", ["temp", "salinity"]);

In [ ]:
summary_df = pd.DataFrame({key: [image[key] for image in images] for key in ("temp", "salinity", "closest_ice", "ice_amount")})
summary_df = summary_df[summary_df["temp"].notna()]
summary_df = summary_df[summary_df["salinity"].notna()]
display(summary_df)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
import matplotlib.ticker as mtick

close_thresh = s2i.cam_props["near_distance"]*4
far_df = summary_df[summary_df["closest_ice"] > close_thresh]

fig, ax = plt.subplots(figsize=(8, 6))
ax.set_ylabel("Ice amount")
ax.set_xscale("log")
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1))
ax.set_xlabel("Distance to closest ice")
plt.scatter(summary_df["closest_ice"], summary_df["ice_amount"]);

As expected, ice amount and distance to closest ice are negatively correlated.

In [ ]:
def plot_temp_salinity(fig : plt.Figure, ax : plt.Axes, temp, salinity, colorby, colorby_name, cmap = "coolwarm",
    # norm=None, t_range = (-2, 11), s_range = (23.75, 34.05)):
    norm=None, t_range = (-2, 6), s_range = (23.75, 33.05)):

    if norm is None: norm = matplotlib.colors.Normalize()
    ax.set_ylim(t_range)
    ax.set_xlim(s_range)
    scatter = ax.scatter(salinity, temp, c=colorby, s=8, cmap=cmap, norm=norm)
    fig.colorbar(scatter, ax=ax, label=colorby_name)
    ax.set_xlabel("Salinity")
    ax.set_ylabel("Temperature (°C)")

fig, ax = plt.subplots(figsize=(8, 6))
plot_temp_salinity(fig, ax, summary_df["temp"], summary_df["salinity"],
    summary_df["closest_ice"], "Distance to Closest Ice (m)", norm=matplotlib.colors.LogNorm(vmax=1000))
ax.set_title("Temperature vs. Salinity, Colored by Distance");

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
plot_temp_salinity(fig, ax, far_df["temp"], far_df["salinity"],
    far_df["closest_ice"], "Distance to Closest Ice (m)", norm=matplotlib.colors.LogNorm(vmax=1000))
ax.set_title("Temp. vs. Sal., Colored by Dist. Excluding Ice Touching Boat");

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
plot_temp_salinity(fig, ax, summary_df["temp"], summary_df["salinity"],
    summary_df["ice_amount"], "Ice Amount (out of 1)")
ax.set_title("Temperature vs. Salinity, Colored by Ice Amount");

In [ ]:
moderate_df = summary_df[(summary_df["ice_amount"] > 0) & (summary_df["ice_amount"] < 1)]
fig, ax = plt.subplots(figsize=(12, 9))
plot_temp_salinity(fig, ax, moderate_df["temp"], moderate_df["salinity"],
    moderate_df["ice_amount"], "Ice Amount (out of 1)")
ax.set_title("Temperature vs. Salinity, Colored by Ice Amount (only non-extreme ice amounts)");

In [ ]:
moderate_far_df = moderate_df[moderate_df["closest_ice"] > close_thresh]
fig, ax = plt.subplots(figsize=(12, 9))
plot_temp_salinity(fig, ax, moderate_far_df["temp"], moderate_far_df["salinity"],
    moderate_far_df["ice_amount"], "Ice Amount (out of 1)")
ax.set_title("Temperature vs. Salinity, Colored by Ice Amount (non-extreme ice excluding touching)");

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 9))
axs = axs.ravel()

axs[0].set_xlabel("closest ice")
axs[0].set_ylabel("temperature")
axs[0].set_xscale("log")
axs[0].scatter(summary_df["closest_ice"], summary_df["temp"], s=2)

axs[1].set_xlabel("closest ice")
axs[1].set_ylabel("salinity")
axs[1].set_xscale("log")
axs[1].scatter(summary_df["closest_ice"], summary_df["salinity"], s=2)

axs[2].set_xlabel("ice amount")
axs[2].set_ylabel("temperature")
axs[2].scatter(summary_df["ice_amount"], summary_df["temp"], s=2)

axs[3].set_xlabel("ice amount")
axs[3].set_ylabel("salinity")
axs[3].scatter(summary_df["ice_amount"], summary_df["salinity"], s=2)

No relationships to speak of between temperature and ice presence or between salinity and ice presence.